<a href="https://colab.research.google.com/github/wy-go/google-research/blob/master/spreadsheet_coder/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data
Load data from [[UI]](https://console.cloud.google.com/storage/browser/spreadsheet_coder)


## Copy files from GCS with gsutil

In [1]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'sheetcoder'
!gcloud config set project {project_id}

Updated property [core/project].


In [2]:
%cd /content/
!ls
!rm -rf google-research 
# Clone the entire repo.
!git clone https://github.com/wy-go/google-research.git

/content
adc.json  google-research  sample_data
Cloning into 'google-research'...
remote: Enumerating objects: 36753, done.
remote: Counting objects: 100% (1442/1442), done.
remote: Compressing objects: 100% (919/919), done.
remote: Total 36753 (delta 630), reused 1132 (delta 498), pack-reused 35311
Receiving objects: 100% (36753/36753), 297.10 MiB | 27.42 MiB/s, done.
Resolving deltas: 100% (19762/19762), done.
Checking out files: 100% (12423/12423), done.


In [3]:
# Install deps
!pip install -q -U tensor2tensor

In [4]:
# Download the file from a given Google Cloud Storage bucket.
%cd google-research/spreadsheet_coder
!gsutil -m cp -r \
  "gs://spreadsheet_coder/enron/" .
!ls

/content/google-research/spreadsheet_coder
Copying gs://spreadsheet_coder/enron/formulas_5_10.tf_record...
Copying gs://spreadsheet_coder/enron/formulas_0_5.tf_record...
Copying gs://spreadsheet_coder/enron/formulas_10_15.tf_record...
Copying gs://spreadsheet_coder/enron/formulas_15_20.tf_record...
Copying gs://spreadsheet_coder/enron/formulas_20_25.tf_record...
Copying gs://spreadsheet_coder/enron/formulas_25_30.tf_record...
| [6/6 files][  3.9 GiB/  3.9 GiB] 100% Done  44.8 MiB/s ETA 00:00:00           
Operation completed over 6 objects/3.9 GiB.                                      
bert_modeling.py	model_utils.py
constants.py		README.md
enron			spreadsheetcoder_model_architecture.png
experiments.ipynb	tokenization.py
mobilebert_modeling.py	vocab.txt
model.py


## Read the TFRecord files

In [5]:
import tensorflow as tf
print(tf.__version__)

tf.gfile = tf.io.gfile

2.6.0


In [6]:
import os

data_dir = 'enron/'
tfrecord_files = [data_dir + filename for filename in os.listdir(data_dir)]
raw_dataset = tf.data.TFRecordDataset(tfrecord_files)

# Show the first 3 records.
for raw_record in raw_dataset.take(3):
  print(repr(raw_record))

<tf.Tensor: shape=(), dtype=string, numpy=b'\n\x8b8\n,\n\x12formula_token_list\x12\x16\n\x14\n\x12b\'=SUM(  RANGE ) \'\n"\n\x06ranges\x12\x18\n\x16\n\x14R[0]C[-12]:R[0]C[-1]\n#\n\x0ecomputed_value\x12\x11\n\x0f\n\rdoub:187875.0\n\x13\n\x06header\x12\t\n\x07\n\x05Total\n\x12\n\tcol_index\x12\x05\x1a\x03\n\x01\r\n\xcb4\n\x0ccontext_data\x12\xba4\n\xb74\n\xb44$R[0]C[-10]$userEnteredValue=doub:8458.0$$R[0]C[-9]$userEnteredValue=doub:7783.0$$R[0]C[-8]$userEnteredValue=doub:9100.0$$R[0]C[-7]$userEnteredValue=doub:85.0$$R[0]C[-6]$userEnteredValue=doub:11240.0$$R[0]C[-5]$userEnteredValue=doub:26013.0$$R[0]C[-4]$userEnteredValue=doub:6065.0$$R[0]C[-3]$userEnteredValue=doub:3388.0$$R[0]C[-2]$userEnteredValue=doub:9964.0$$R[0]C[-1]$userEnteredValue=doub:51265.0$$R[0]C[0]$formula=RANGE SUM FORMULA_START, formulaTokenList==SUM( RANGE ), ranges=[R1C1FormulaRange{relativeRange=R[0]C[-12]:R[0]C[-1], startTokenIndex=3, workbookRangeId=null}], computedValue=doub:187875.0$$R[1]C[-10]$userEnteredValue=dou

In [7]:
# Create a description of the features.
feature_description = {
    'table_id': tf.io.FixedLenFeature([], tf.int64),
    'doc_id': tf.io.FixedLenFeature([], tf.string),
    'record_index': tf.io.FixedLenFeature([], tf.int64),
    'col_index': tf.io.FixedLenFeature([], tf.int64),
    'formula': tf.io.FixedLenFeature([], tf.string),
    'formula_token_list': tf.io.FixedLenFeature([], tf.string),
    'ranges': tf.io.FixedLenFeature([], tf.string),
    'computed_value': tf.io.FixedLenFeature([], tf.string),
    'header': tf.io.FixedLenFeature([], tf.string),
    'context_header': tf.io.FixedLenFeature([], tf.string),
    'context_data': tf.io.FixedLenFeature([], tf.string)
}

def _parse_example(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  feature_dict = tf.io.parse_single_example(example_proto, feature_description)
  return feature_dict

parsed_dataset = raw_dataset.map(_parse_example)

parsed_dataset

<MapDataset shapes: {col_index: (), computed_value: (), context_data: (), context_header: (), doc_id: (), formula: (), formula_token_list: (), header: (), ranges: (), record_index: (), table_id: ()}, types: {col_index: tf.int64, computed_value: tf.string, context_data: tf.string, context_header: tf.string, doc_id: tf.string, formula: tf.string, formula_token_list: tf.string, header: tf.string, ranges: tf.string, record_index: tf.int64, table_id: tf.int64}>

In [8]:
# Show the first 3 observations in datasets.
for parsed_record in parsed_dataset.take(10):
  print(parsed_record)

{'col_index': <tf.Tensor: shape=(), dtype=int64, numpy=13>, 'computed_value': <tf.Tensor: shape=(), dtype=string, numpy=b'doub:187875.0'>, 'context_data': <tf.Tensor: shape=(), dtype=string, numpy=b'$R[0]C[-10]$userEnteredValue=doub:8458.0$$R[0]C[-9]$userEnteredValue=doub:7783.0$$R[0]C[-8]$userEnteredValue=doub:9100.0$$R[0]C[-7]$userEnteredValue=doub:85.0$$R[0]C[-6]$userEnteredValue=doub:11240.0$$R[0]C[-5]$userEnteredValue=doub:26013.0$$R[0]C[-4]$userEnteredValue=doub:6065.0$$R[0]C[-3]$userEnteredValue=doub:3388.0$$R[0]C[-2]$userEnteredValue=doub:9964.0$$R[0]C[-1]$userEnteredValue=doub:51265.0$$R[0]C[0]$formula=RANGE SUM FORMULA_START, formulaTokenList==SUM( RANGE ), ranges=[R1C1FormulaRange{relativeRange=R[0]C[-12]:R[0]C[-1], startTokenIndex=3, workbookRangeId=null}], computedValue=doub:187875.0$$R[1]C[-10]$userEnteredValue=doub:8353.0$$R[1]C[-9]$userEnteredValue=doub:7686.0$$R[1]C[-8]$userEnteredValue=doub:8986.0$$R[1]C[-7]$userEnteredValue=doub:84.0$$R[1]C[-6]$userEnteredValue=doub:

  <table border="1">
  <caption><i>Table 1. </i>Sample Data</caption>
    <thead>
      <tr>
        <th></th>
        <th>table_id</th>
        <th>doc_id</th>
        <th>record_index</th>
        <th>col_index</th>
        <th>formula</th>
        <th>formula_token_list</th>
        <th>ranges</th>
        <th>computed_value</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>1</th>
        <td>2</td>
        <td>b''</td>
        <td>1</td>
        <td>19</td>
        <td>b'RANGE UPLUS FORMULA_START'</td>
        <td>b"b'= +  RANGE'"</td>
        <td>b'G278457581!R[3]C[13]'</td>
        <td>b'doub:83.41666666666667'</td>
      </tr>
      <tr align=center>
        <th align=left>2</th>
        <td>2</td>
        <td>b''</td>
        <td>1</td>
        <td>20</td>
        <td>b'RANGE UPLUS FORMULA_START'</td>
        <td>b"b'= +  RANGE'"</td>
        <td>b'G278457581!R[4]C[12]'</td>
        <td>b'doub:54.220833333333346'</td>
      </tr>
      <tr align=center>
        <th align=left>3</th>
        <td>2</td>
        <td>b''</td>
        <td>1</td>
        <td>21</td>
        <td>b'RANGE UPLUS FORMULA_START'</td>
        <td>b"b'= +  RANGE'"</td>
        <td>b'G278457581!R[3]C[-17]'</td>
        <td>b'doub:49.215833333333336'</td>
      </tr>
    </tbody>
  </table>

<table border="1">
    <thead>
      <tr>
        <th></th>
        <th>header</th>
        <th>context_header</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>1</th>
        <td>b'Contract Cost'</td>
        <td>b'$C[-10]$New/Used$C[-9]$Delivery Date$C[-8]$Owner$C[-7]$DASH\nApproval\nDate$C[-6]$Notice to Proceed Given<br/>$C[-5]$Status of Financing$C[-4]$Controlled By$C[-3]$Originator / Developer$C[-2]$Project Manager$C[-1]$Project Name<br/>$C[0]$Contract Cost$C[1]$Contract Paid to Date\n(Scheduled)$C[2]$\nCancel-lation Payment$C[3]$Comments$C[4]$Status Update'</td>
      </tr>
      <tr align=center>
        <th align=left>2</th>
        <td>b'Contract Paid to Date\n(Scheduled)'</td>
        <td>b'$C[-10]$Delivery Date$C[-9]$Owner$C[-8]$DASH\nApproval\nDate$C[-7]$Notice to Proceed Given$C[-6]$Status of Financing<br/>$C[-5]$Controlled By$C[-4]$Originator / Developer$C[-3]$Project Manager$C[-2]$Project Name$C[-1]$Contract Cost<br/>$C[0]$Contract Paid to Date\n(Scheduled)$C[1]$\nCancel-lation Payment$C[2]$Comments$C[3]$Status Update'</td>
      </tr>
      <tr align=center>
        <th align=left>3</th>
        <td>b'\nCancel-lation Payment'</td>
        <td>b'$C[-10]$Owner$C[-9]$DASH\nApproval\nDate$C[-8]$Notice to Proceed Given$C[-7]$Status of Financing$C[-6]$Controlled By<br/>$C[-5]$Originator / Developer$C[-4]$Project Manager$C[-3]$Project Name$C[-2]$Contract Cost$C[-1]$Contract Paid to Date\n(Scheduled)<br/>$C[0]$\nCancel-lation Payment$C[1]$Comments$C[2]$Status Update'</td>
      </tr>
    </tbody>
<table>

<table border="1">
    <thead>
      <tr>
        <th></th>
        <th>context_data</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>1</th>
        <td>b'$R[-1]C[-10]$userEnteredValue=empty$<br/>$R[-1]C[-9]$userEnteredValue=empty$<br/>$R[-1]C[-8]$userEnteredValue=empty$<br/>$R[-1]C[-7]$userEnteredValue=empty$<br/>$R[-1]C[-6]$userEnteredValue=empty$<br/>$R[-1]C[-5]$userEnteredValue=empty$<br/>$R[-1]C[-4]$userEnteredValue=empty$<br/>$R[-1]C[-3]$userEnteredValue=empty$<br/>$R[-1]C[-2]$userEnteredValue=empty$<br/>$R[-1]C[-1]$userEnteredValue=empty$<br/>$R[-1]C[0]$userEnteredValue=empty$<br/>$R[-1]C[1]$userEnteredValue=empty$<br/>$R[-1]C[2]$userEnteredValue=empty$<br/>$R[-1]C[3]$userEnteredValue=empty$<br/>$R[-1]C[4]$userEnteredValue=empty$<br/>$R[0]C[-10]$userEnteredValue=str:New$<br/>$R[0]C[-9]$userEnteredValue=doub:37165.0$<br/>$R[0]C[-8]$userEnteredValue=str:Whitewing$<br/>$R[0]C[-7]$userEnteredValue=str:$4.5MM DASHed$<br/>$R[0]C[-6]$userEnteredValue=str:N$<br/>$R[0]C[-5]$userEnteredValue=str:N/A$<br/>$R[0]C[-4]$userEnteredValue=str:EWS$<br/>$R[0]C[-3]$userEnteredValue=str:John Chappell$<br/>$R[0]C[-2]$userEnteredValue=str:Stephen Heck$<br/>$R[0]C[-1]$userEnteredValue=str:Sale in Process$<br/>$R[0]C[0]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[3]C[-17], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:83.41666666666667$<br/>$R[0]C[1]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[3]C[13], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:54.220833333333346$<br/>$R[0]C[2]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[4]C[12], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:49.215833333333336$<br/>$R[0]C[3]$userEnteredValue=empty$<br/>$R[0]C[4]$userEnteredValue=str:DASH in progress.$<br/>$R[1]C[-10]$userEnteredValue=str:New$<br/>$R[1]C[-9]$userEnteredValue=doub:37196.0$<br/>$R[1]C[-8]$userEnteredValue=str:Whitewing$<br/>$R[1]C[-7]$userEnteredValue=str:$4.5MM DASHed$<br/>$R[1]C[-6]$userEnteredValue=str:N$<br/>$R[1]C[-5]$userEnteredValue=str:N/A$<br/>$R[1]C[-4]$userEnteredValue=str:EWS$<br/>$R[1]C[-3]$userEnteredValue=str:John Chappell$<br/>$R[1]C[-2]$userEnteredValue=str:Stephen Heck$<br/>$R[1]C[-1]$userEnteredValue=str:Sale in Process$<br/>$R[1]C[0]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[10]C[-17], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:83.41666666666667$<br/>$R[1]C[1]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[10]C[13], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:54.220833333333346$<br/>$R[1]C[2]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[11]C[12], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:49.215833333333336$<br/>$R[1]C[3]$userEnteredValue=empty$<br/>$R[1]C[4]$userEnteredValue=str:DASH in progress.$<br/>$R[2]C[-10]$userEnteredValue=str:New$<br/>$R[2]C[-9]$userEnteredValue=doub:37226.0$<br/>$R[2]C[-8]$userEnteredValue=str:Whitewing$<br/>$R[2]C[-7]$userEnteredValue=str:$4.5MM DASHed$<br/>$R[2]C[-6]$userEnteredValue=str:N$<br/>$R[2]C[-5]$userEnteredValue=str:N/A$<br/>$R[2]C[-4]$userEnteredValue=str:EWS$<br/>$R[2]C[-3]$userEnteredValue=str:John Chappell$<br/>$R[2]C[-2]$userEnteredValue=str:Stephen Heck$<br/>$R[2]C[-1]$userEnteredValue=str:Sale in Process$<br/>$R[2]C[0]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[17]C[-17], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:83.41666666666667$<br/>$R[2]C[1]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[17]C[13], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:54.220833333333346$<br/>$R[2]C[2]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[18]C[12], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:49.215833333333336$<br/>$R[2]C[3]$userEnteredValue=empty$<br/>$R[2]C[4]$userEnteredValue=str:DASH in progress.$<br/>$R[3]C[-10]$userEnteredValue=str:New$<br/>$R[3]C[-9]$userEnteredValue=doub:37135.0$<br/>$R[3]C[-8]$userEnteredValue=str:ENE B/S$<br/>$R[3]C[-7]$userEnteredValue=str:Approved$<br/>$R[3]C[-6]$userEnteredValue=str:N$<br/>$R[3]C[-5]$userEnteredValue=str:N/A$<br/>$R[3]C[-4]$userEnteredValue=str:EWS$<br/>$R[3]C[-3]$userEnteredValue=str:Dick Westfahl$<br/>$R[3]C[-2]$userEnteredValue=empty$<br/>$R[3]C[-1]$userEnteredValue=str:NEPCO / NESCO - Goldendale (EECC)$<br/>$R[3]C[0]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[32]C[-17], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:36.24736$<br/>$R[3]C[1]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[32]C[13], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:35.613031199999995$<br/>$R[3]C[2]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[33]C[12], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:35.7036496$<br/>$R[3]C[3]$userEnteredValue=empty$<br/>$R[3]C[4]$userEnteredValue=str:Contract in the works, possible buyer$<br/>$R[4]C[-10]$userEnteredValue=str:Used$<br/>$R[4]C[-9]$userEnteredValue=str:Delivered$<br/>$R[4]C[-8]$userEnteredValue=str:West LB$<br/>$R[4]C[-7]$userEnteredValue=str:Analyzing$<br/>$R[4]C[-6]$userEnteredValue=str:N$<br/>$R[4]C[-5]$userEnteredValue=str:N/A$<br/>$R[4]C[-4]$userEnteredValue=str:EA$<br/>$R[4]C[-3]$userEnteredValue=empty$<br/>$R[4]C[-2]$userEnteredValue=empty$<br/>$R[4]C[-1]$userEnteredValue=str:Sale in Process$<br/>$R[4]C[0]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[111]C[-17], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:17.25$<br/>$R[4]C[1]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[111]C[13], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:17.25$<br/>$R[4]C[2]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[112]C[12], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:17.25$<br/>$R[4]C[3]$userEnteredValue=empty$<br/>$R[4]C[4]$userEnteredValue=str:3 Potential buyers in Canada, no legitimate offer as of 1/25/01.$<br/>$R[5]C[-10]$userEnteredValue=str:Used$<br/>$R[5]C[-9]$userEnteredValue=str:Delivered$<br/>$R[5]C[-8]$userEnteredValue=str:West LB$<br/>$R[5]C[-7]$userEnteredValue=str:Analyzing$<br/>$R[5]C[-6]$userEnteredValue=str:N$<br/>$R[5]C[-5]$userEnteredValue=str:N/A$<br/>$R[5]C[-4]$userEnteredValue=str:EA$<br/>$R[5]C[-3]$userEnteredValue=empty$<br/>$R[5]C[-2]$userEnteredValue=empty$<br/>$R[5]C[-1]$userEnteredValue=str:Sale in Process$<br/>$R[5]C[0]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[118]C[-17], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:17.25$<br/>$R[5]C[1]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[118]C[13], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:17.25$<br/>$R[5]C[2]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[119]C[12], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:17.25$<br/>$R[5]C[3]$userEnteredValue=empty$<br/>$R[5]C[4]$userEnteredValue=str:3 Potential buyers in Canada, no legitimate offer as of 1/25/01.$<br/>$R[6]C[-10]$userEnteredValue=str:New$<br/>$R[6]C[-9]$userEnteredValue=str:being cleaned$<br/>$R[6]C[-8]$userEnteredValue=str:ENE B/S$<br/>$R[6]C[-7]$userEnteredValue=str:Analyzing$<br/>$R[6]C[-6]$userEnteredValue=str:N$<br/>$R[6]C[-5]$userEnteredValue=str:N/A$<br/>$R[6]C[-4]$userEnteredValue=str:EA$<br/>$R[6]C[-3]$userEnteredValue=str:David Fairley, Mathew Gimble$<br/>$R[6]C[-2]$userEnteredValue=empty$<br/>$R[6]C[-1]$userEnteredValue=str:Purchaser Identified$<br/>$R[6]C[0]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[21]C[-17], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:24.506$<br/>$R[6]C[1]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[21]C[13], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:24.506000000000007$<br/>$R[6]C[2]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[22]C[12], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:24.506$<br/>$R[6]C[3]$userEnteredValue=str:CALME purchased turbine from ENA; turbine has not yet cleared customs;  generator incurred salt water damage while unloading$<br/>$R[6]C[4]$userEnteredValue=empty$<br/>$R[7]C[-10]$userEnteredValue=empty$<br/>$R[7]C[-9]$userEnteredValue=empty$<br/>$R[7]C[-8]$userEnteredValue=str:E-Next Generation$<br/>$R[7]C[-7]$userEnteredValue=str:$16.5MM on 2/16/01$<br/>$R[7]C[-6]$userEnteredValue=str:N$<br/>$R[7]C[-5]$userEnteredValue=str:N/A$<br/>$R[7]C[-4]$userEnteredValue=str:EA$<br/>$R[7]C[-3]$userEnteredValue=str:Jake Thomas/Laura Wente$<br/>$R[7]C[-2]$userEnteredValue=empty$<br/>$R[7]C[-1]$userEnteredValue=str:Columbia$<br/>$R[7]C[0]$formula=RANGE FORMULA_START, formulaTokenList== RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[100]C[-17], startTokenIndex=1, workbookRangeId=null}], computedValue=doub:39.2$<br/>$R[7]C[1]$formula=RANGE FORMULA_START, formulaTokenList== RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[100]C[13], startTokenIndex=1, workbookRangeId=null}], computedValue=doub:6.272$<br/>$R[7]C[2]$formula=RANGE FORMULA_START, formulaTokenList== RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[101]C[12], startTokenIndex=1, workbookRangeId=null}], computedValue=doub:4.704$<br/>$R[7]C[3]$userEnteredValue=empty$<br/>$R[7]C[4]$userEnteredValue=empty$<br/>$R[8]C[-10]$userEnteredValue=str:New$<br/>$R[8]C[-9]$userEnteredValue=doub:37408.0$<br/>$R[8]C[-8]$userEnteredValue=str:West LB$<br/>$R[8]C[-7]$userEnteredValue=str:$2.5MM on 1/31/01$<br/>$R[8]C[-6]$userEnteredValue=str:N$<br/>$R[8]C[-5]$userEnteredValue=str:N/A$<br/>$R[8]C[-4]$userEnteredValue=str:EA$<br/>$R[8]C[-3]$userEnteredValue=empty$<br/>$R[8]C[-2]$userEnteredValue=empty$<br/>$R[8]C[-1]$userEnteredValue=str:Fort Pierce$<br/>$R[8]C[0]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[139]C[-17], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:43.618$<br/>$R[8]C[1]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[139]C[13], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:34.894400000000005$<br/>$R[8]C[2]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[140]C[12], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:43.618$<br/>$R[8]C[3]$userEnteredValue=empty$<br/>$R[8]C[4]$userEnteredValue=empty$<br/>$R[9]C[-10]$userEnteredValue=str:New$<br/>$R[9]C[-9]$userEnteredValue=doub:37591.0$<br/>$R[9]C[-8]$userEnteredValue=empty$<br/>$R[9]C[-7]$userEnteredValue=str:$4.2MM on 7/24/00$<br/>$R[9]C[-6]$userEnteredValue=str:N$<br/>$R[9]C[-5]$userEnteredValue=str:N/A$<br/>$R[9]C[-4]$userEnteredValue=str:EA$<br/>$R[9]C[-3]$userEnteredValue=str:Maurice Gilbert$<br/>$R[9]C[-2]$userEnteredValue=empty$<br/>$R[9]C[-1]$userEnteredValue=str:Las Vegas CoGen II$<br/>$R[9]C[0]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[34]C[-17], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:15.769725$<br/>$R[9]C[1]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[34]C[13], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:3.94243125$<br/>$R[9]C[2]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[35]C[12], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:3.153945$<br/>$R[9]C[3]$userEnteredValue=empty$<br/>$R[9]C[4]$userEnteredValue=empty$<br/>$R[10]C[-10]$userEnteredValue=str:New$<br/>$R[10]C[-9]$userEnteredValue=doub:37257.0$<br/>$R[10]C[-8]$userEnteredValue=empty$<br/>$R[10]C[-7]$userEnteredValue=str:$4.2MM on 7/24/00$<br/>$R[10]C[-6]$userEnteredValue=str:N$<br/>$R[10]C[-5]$userEnteredValue=str:N/A$<br/>$R[10]C[-4]$userEnteredValue=str:EA$<br/>$R[10]C[-3]$userEnteredValue=str:Maurice Gilbert$<br/>$R[10]C[-2]$userEnteredValue=empty$<br/>$R[10]C[-1]$userEnteredValue=str:Las Vegas CoGen II$<br/>$R[10]C[0]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[41]C[-17], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:15.769725$<br/>$R[10]C[1]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[41]C[13], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:3.94243125$<br/>$R[10]C[2]$formula=RANGE UPLUS FORMULA_START, formulaTokenList==+ RANGE, ranges=[R1C1FormulaRange{relativeRange=G278457581!R[42]C[12], startTokenIndex=2, workbookRangeId=null}], computedValue=doub:3.153945$<br/>$R[10]C[3]$userEnteredValue=empty$<br/>$R[10]C[4]$userEnteredValue=empty$'
      </tr>
    </tbody>
<table>

## Data preprocess
```
record_index: tf.int64
col_index: tf.int64
formula: tf.string, '<SKETCH> $ENDSKETCH$ $R$ R[]C[] $RANGESPLIT$ R[]C[] $ENDFORMULASKETCH$ EOF'
context_header: tf.string
context_data: tf.string, [21, 21]
```

In [9]:
import re
import numpy as np
import tensorflow_datasets as tfds
from constants import SPECIAL_TOKEN_LIST, END_FORMULA_SKETCH_ID

D = 10
print(SPECIAL_TOKEN_LIST)

['PAD', 'EOF', 'UNK', 'GO', 'empty', '$RANGESPLIT$', '$ENDRANGE$', '$ENDFORMULASKETCH$', 'R[0]', 'R[1]', 'R[2]', 'R[3]', 'R[4]', 'R[5]', 'R[6]', 'R[7]', 'R[8]', 'R[9]', 'R[10]', 'R[-1]', 'R[-2]', 'R[-3]', 'R[-4]', 'R[-5]', 'R[-6]', 'R[-7]', 'R[-8]', 'R[-9]', 'R[-10]', 'C[0]', 'C[1]', 'C[2]', 'C[3]', 'C[4]', 'C[5]', 'C[6]', 'C[7]', 'C[8]', 'C[9]', 'C[10]', 'C[-1]', 'C[-2]', 'C[-3]', 'C[-4]', 'C[-5]', 'C[-6]', 'C[-7]', 'C[-8]', 'C[-9]', 'C[-10]', 'str:', 'doub:', 'bool:', 'quotedstr:', 'err:', 'image:', 'sparkchart:']


In [10]:
# Complete brackets, eliminate Gxxxx
def format_range(range_instance):
  range_eliminate = re.sub('G\d*', '', range_instance)
  range_eliminate = re.sub('"', '', range_eliminate)
  str_list = list(range_eliminate)
  for i in range(len(str_list) + 8):
    if i >= len(str_list):
      break
    if (str_list[i] == 'R' or str_list[i] == 'C') and str_list[i + 1] != '[':
      str_list.insert(i + 1, '[')
    if (str_list[i] == 'C' or str_list[i] == ':') and str_list[i - 1] != ']':
        str_list.insert(i, ']')
  if str_list[-1] != ']':
    str_list += ']'

  format_string = ''.join(str_list)
  range_values = np.array([int(range_string[1:-1]) for range_string in re.findall('\[.*?\]', format_string)])
  exceed = not ((-D <= range_values) & (range_values < 11)).all()

  return format_string, exceed

a, b = format_range('G88601434R-11C5:R[3]C[9"')
print(a, b)

R[-11]C[5]:R[3]C[9] True


In [11]:
#
def format_example(example):

  formula_token_list = str(example['formula_token_list'].numpy())
  ranges = str(example['ranges'].numpy())
  context_header = str(example['context_header'].numpy())
  context_data = str(example['context_data'].numpy())


  sketch = re.search("b'(.*)'", formula_token_list).group(1)
  range_simple = re.search("b'(.*)'", ranges)
  if range_simple:
    range_list = range_simple.group(1).split(' ')
  else:
    range_list = re.findall('!(.*?) ', ranges) + ranges.split("!")[-1:]
  exceed = False
  format_ranges = []
  for ra in range_list:
    format_ra, exceed = format_range(ra)
    format_ranges.append(format_ra)
    if exceed:
      break
  if exceed:
    return None

  formula = sketch + ' $ENDFORMULASKETCH$ $R$ '
  for (i, range_i) in enumerate(format_ranges):
    if(i):
      formula += ' $RANGESPLIT$ '
    formula += range_i
  formula += ' $ENDRANGE$ EOF'
  
  tokenizer = tfds.deprecated.text.Tokenizer(
      alphanum_only=False, reserved_tokens=SPECIAL_TOKEN_LIST + ['$R$'])
  formula_tokens = tokenizer.tokenize(formula.replace(' ', ''))
  formula_doc.append(formula_tokens)
  
  cells = context_header[3:-1].split('$')

  if len(cells):
    start_col_index = int(cells[0][2:-1])
    for i in range(len(cells)):
      if i >= len(cells):
        break
      while i % 2 == 0 and cells[i] != "C[" + str(start_col_index + i // 2) + "]":
        part_of_former = cells[i]
        cells.pop(i)
        if i > 0:
          cells[i - 1] += '$' + part_of_former
        if i >= len(cells):
          break
      if i >= len(cells):
        break
        
    end_col_index = int(cells[-2][2:-1])
  else:
    start_col_index = 10
    end_col_index = 10

  context_header = []
  if start_col_index > -D:
    context_header += ['empty'] * (start_col_index + D)

  for i in range(end_col_index - start_col_index + 1):
    if cells[2 * i + 1]:
      context_header += [cells[2 * i + 1]]
    else:
      context_header += ['empty']

  if end_col_index < D:
    context_header += ['empty'] * (D - end_col_index)


  cells = context_data[3:-1].split('$')
    
  context_data = ['empty'] * 21 * 21
  cell_num = len(cells) // 3
  last_index = 0

  for i in range(cell_num):

    if len(cells) <= 3 * i:
      break;
    cell_index = re.search("R\[(.*)\]C\[(.*)\]", cells[3 * i])
    while not cell_index:
      part_of_last = cells[3 * i - 1]
      cells.pop(3 * i - 1)
      context_data[last_index] += '$' + part_of_last
      if 3 * i >= len(cells):
        break
      cell_index = re.search("R\[(.*)\]C\[(.*)\]", cells[3 * i])
    if not cell_index:
      break

    row_index = int(cell_index.group(1))
    col_index = int(cell_index.group(2))
    context_index = (row_index + 10) * 21 + col_index + 10
    last_index = context_index

    value = cells[3 * i + 1]
    value = re.search("Value=(.*)", value).group(1)

    context_data[context_index] = value

  formula = tf.convert_to_tensor(np.array(formula),
                                         dtype=tf.string)
  context_header = tf.convert_to_tensor(np.array(context_header),
                                         dtype=tf.string)
  context_data = tf.convert_to_tensor(np.array(context_data).reshape(21, 21),
                                         dtype=tf.string)
  return {'record_index': example['record_index'], 
          'col_index': example['col_index'],
          'formula': formula, 
          'context_header': context_header,
          'context_data': context_data
          }

format_dataset = []
formula_doc = []
ORIG_SIZE = 0
for parsed_data in parsed_dataset:
  format_data = format_example(parsed_data)
  ORIG_SIZE +=1
  if format_data:
    format_dataset.append(format_data)


In [12]:
# Show observations in datasets.
for format_record in format_dataset[1:2]:
  print(repr(format_record))

{'record_index': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'col_index': <tf.Tensor: shape=(), dtype=int64, numpy=7>, 'formula': <tf.Tensor: shape=(), dtype=string, numpy=b'= RANGE *  RANGE $ENDFORMULASKETCH$ $R$ R[0]C[-4] $RANGESPLIT$ R[0]C[-2] $ENDRANGE$ EOF'>, 'context_header': <tf.Tensor: shape=(21,), dtype=string, numpy=
array([b'empty', b'empty', b'empty', b'empty', b'empty', b'empty',
       b'empty', b'empty', b'empty', b'empty', b'empty', b'empty',
       b'empty', b'empty', b'empty', b'empty', b'empty', b'empty',
       b'empty', b'empty', b'empty'], dtype=object)>, 'context_data': <tf.Tensor: shape=(21, 21), dtype=string, numpy=
array([[b'empty', b'empty', b'empty', b'empty', b'empty', b'empty',
        b'empty', b'empty', b'empty', b'empty', b'empty', b'empty',
        b'empty', b'empty', b'empty', b'empty', b'empty', b'empty',
        b'empty', b'empty', b'empty'],
       [b'empty', b'empty', b'empty', b'empty', b'empty', b'empty',
        b'empty', b'empty', b'empty', b

## Construct output vocabulary

In [13]:
from collections import defaultdict

class Vocab:
  def __init__(self, tokens=None):
    self.idx_to_token = list()
    self.token_to_idx = dict()

    if tokens is not None:
      if "UNK" not in tokens:
        tokens = tokens + ["UNK"]
      for token in tokens:
        self.idx_to_token.append(token)
        self.token_to_idx[token] = len(self.idx_to_token) - 1
      self.unk = self.token_to_idx['UNK']

  @classmethod
  def build(cls, text, min_freq=1, reserved_tokens=None):
    token_freqs = defaultdict(int)
    for sentence in text:
      for token in sentence:
        token_freqs[token] += 1
    token_list = reserved_tokens if reserved_tokens else []
    uniq_tokens = set(token_list)
    for token, freq in token_freqs.items():
      if freq >= min_freq and (token not in uniq_tokens):
        token_list += [token]
        uniq_tokens.add(token)
    return cls(tokens=token_list)
  
  def __len__(self):
    return len(self.idx_to_token)

  def __getitem__(self, token):
    return self.token_to_idx.get(token, self.unk)
  
  def convert_tokens_to_ids(self, tokens):
    return [self[token] for token in tokens]

  def convert_ids_to_tokens(self, indices):
    return [self.idx_to_token[index] for index in indices]

In [14]:
# Filter out tokens that appear less than 10 times in the training set to construct the output formula token vocabulary.
reserver_tokens = list(SPECIAL_TOKEN_LIST) + ['$R$']
output_vocab = Vocab.build(formula_doc, min_freq=10, reserved_tokens=reserver_tokens)
VOCAB_SIZE = len(output_vocab)

print(output_vocab.idx_to_token)
print(VOCAB_SIZE)

['PAD', 'EOF', 'UNK', 'GO', 'empty', '$RANGESPLIT$', '$ENDRANGE$', '$ENDFORMULASKETCH$', 'R[0]', 'R[1]', 'R[2]', 'R[3]', 'R[4]', 'R[5]', 'R[6]', 'R[7]', 'R[8]', 'R[9]', 'R[10]', 'R[-1]', 'R[-2]', 'R[-3]', 'R[-4]', 'R[-5]', 'R[-6]', 'R[-7]', 'R[-8]', 'R[-9]', 'R[-10]', 'C[0]', 'C[1]', 'C[2]', 'C[3]', 'C[4]', 'C[5]', 'C[6]', 'C[7]', 'C[8]', 'C[9]', 'C[10]', 'C[-1]', 'C[-2]', 'C[-3]', 'C[-4]', 'C[-5]', 'C[-6]', 'C[-7]', 'C[-8]', 'C[-9]', 'C[-10]', 'str:', 'doub:', 'bool:', 'quotedstr:', 'err:', 'image:', 'sparkchart:', '$R$', '=', 'AVERAGE', '(', 'RANGE', ')', ':', '*', '+', '=+', 'SUM', '-', '/', '!', '=+(', ')*', '*(', '=(', ')-', '/(', 'WEEKDAY', ')+', 'COUNTA', ')+(', '=(-', '=-', ')/', '=(+', '=((', '+(', '))', '=-(', 'REF', '!+#', 'DAY', 'LN', 'DATEVALUE', 'ABS', 'YEAR', 'MONTH', 'COUNT', '-(', '(+', ')/(', '=+((', 'b', "\\'=", '&":"&', "\\'"]
106


## Statistics

### From paper

  <table border="1">
  <caption><i>Table 1. </i>Dataset</caption>
    <thead>
      <tr>
        <th></th>
        <th>Train</th>
        <th>Validation</th>
        <th>Test</th>
        <th>Total</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>#Samples</th>
        <td>178K</td>
        <td>41K</td>
        <td>33K</td>
        <td>252K</td>
      </tr>
    </tbody>
  </table>

  <br />

  <table border="1">
  <caption><i>Table 2. </i>Sketch Length (excluding ENDSKETCH).</caption>
    <thead>
      <tr>
        <th align=left>Length</th>
        <th>2</th>
        <th>3</th>
        <th>4-5</th>
        <th>6-7</th>
        <th>8+</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>Distribution</th>
        <td>55%</td>
        <td>18%</td>
        <td>13%</td>
        <td>9%</td>
        <td>5%</td>
      </tr>
    </tbody>
  </table>

  <br />
  
  <table border="1">
  <caption><i>Table 3. </i>Spreadsheet Functions & Operators</caption>
    <thead>
      <tr>
        <th></th>
        <th>Spreadsheet Functions</th>
        <th>Type</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>1</th>
        <td>ADD(+)</td>
        <td>Operator</td>
      </tr>
      <tr align=center>
        <th align=left>2</th>
        <td>MINUS(-)</td>
        <td>Operator</td>
      </tr>
      <tr align=center>
        <th align=left>3</th>
        <td>MULPTIPLY(*)</td>
        <td>Operator</td>
      </tr>
      <tr align=center>
        <th align=left>4</th>
        <td>DIV(/)</td>
        <td>Operator</td>
      </tr>
      <tr align=center>
        <th align=left>5</th>
        <td>UPLUS</td>
        <td>Operator</td>
      </tr>
      <tr align=center>
        <th align=left>6</th>
        <td>UMINUS</td>
        <td>Operator</td>
      </tr>      
      <tr align=center>
        <th align=left>7</th>
        <td>SUM</td>
        <td>Math</td>
      </tr>
      <tr align=center>
        <th align=left>8</th>
        <td>ABS</td>
        <td>Math</td>
      </tr>
      <tr align=center>
        <th align=left>9</th>
        <td>LN</td>
        <td>Math</td>
      </tr>
      <tr align=center>
        <th align=left>10</th>
        <td>AVERAGE</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>11</th>
        <td>MIN</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>12</th>
        <td>MAX</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>13</th>
        <td>COUNT</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>14</th>
        <td>COUNTA</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>15</th>
        <td>STDEV</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>16</th>
        <td>DAY</td>
        <td>Date</td>
      </tr>
      <tr align=center>
        <th align=left>17</th>
        <td>WEEKDAY</td>
        <td>Date</td>
      </tr>
    </tbody>
  </table>

<br />

### From data

In [15]:
import tokenization

TOTAL_SIZE = len(format_dataset)
print("original dataset size: %d" %(ORIG_SIZE))
print("dataset size: %d" %(TOTAL_SIZE))

formula_len = defaultdict(int)
sketch_len = defaultdict(int)

formula_ids = []
for i, data in enumerate(format_dataset):
  formula = output_vocab.convert_tokens_to_ids(formula_doc[i])
  formula_ids.append(formula)
  formula_len[len(formula)] += 1
  sketch_len[formula.index(END_FORMULA_SKETCH_ID)] += 1

print("full formula length distribution:")
print(formula_len)
print("sketch length distribution (excluding $END_FORMULA_SKETCH$):")
print(sketch_len)


original dataset size: 251224
dataset size: 172908
full formula length distribution:
defaultdict(<class 'int'>, {14: 43915, 13: 42956, 16: 2089, 18: 6119, 8: 42225, 9: 1522, 28: 275, 10: 31, 11: 2744, 23: 19559, 19: 1985, 15: 1668, 20: 198, 21: 664, 63: 1788, 69: 385, 87: 1422, 75: 672, 24: 372, 81: 222, 43: 159, 33: 55, 12: 637, 38: 32, 27: 59, 35: 1, 29: 20, 31: 29, 36: 10, 76: 1, 99: 480, 45: 17, 39: 1, 57: 596})
sketch length distribution (excluding $END_FORMULA_SKETCH$):
defaultdict(<class 'int'>, {5: 47197, 4: 44445, 7: 5979, 6: 4893, 2: 43682, 10: 295, 8: 19817, 9: 663, 20: 1788, 22: 385, 28: 1422, 24: 672, 26: 222, 3: 66, 16: 159, 12: 35, 14: 49, 23: 1, 11: 21, 13: 29, 15: 11, 31: 1, 32: 480, 18: 596})


In [16]:
# Observe
for i in [10, 210, 350]:
  print(formula_doc[i], formula_ids[i])

['=', 'RANGE', '+', 'RANGE', '$ENDFORMULASKETCH$', '$R$', 'R[0]', 'C[-2]', '$RANGESPLIT$', 'R[0]', 'C[-1]', '$ENDRANGE$', 'EOF'] [58, 61, 65, 61, 7, 57, 8, 41, 5, 8, 40, 6, 1]
['=+', 'RANGE', '$ENDFORMULASKETCH$', '$R$', '!', 'R[3]', 'C[0]', '$ENDRANGE$', 'EOF'] [66, 61, 7, 57, 70, 11, 29, 6, 1]
['=+', 'RANGE', '$ENDFORMULASKETCH$', '$R$', 'R[2]', 'C[-3]', '$ENDRANGE$', 'EOF'] [66, 61, 7, 57, 10, 42, 6, 1]


## Split dataset and save as TFRecord

In [42]:
VOCAB_FILE = 'vocab.txt'
FORMULA_LENGTH = 100 # not provided in paper

def _truncate_cell_index(len_list, max_len):
  st_idx, end_idx = (0, 21)
  total_len = sum(len_list)
  while total_len > max_len:
    if end_idx - 11 == 10 - st_idx:
      end_idx -= 1
      total_len -= len_list[end_idx]
    else:
      st_idx += 1
      total_len -= len_list[st_idx - 1]
  return st_idx, end_idx

def row_context_builder(context_data, context_header):
  if not isinstance(context_data, list):
    context_data = context_data.tolist()
    context_header = context_header.tolist()
  context_data = [context_header] + context_data
  row_context = []
  pad_lens = []
  cell_ranges = []
  cell_indices = []
  for i in range(22):
    len_list = []
    for j in range(21):
      tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=True)
      context =  tokenization.convert_to_unicode(context_data[i][j])
      tokens = tokenizer.tokenize(context) + ["[SEP]"]
      tokens = tokenizer.convert_tokens_to_ids(tokens)
      len_list.append(len(tokens))
      context_data[i][j] = tokens
    cur_line = []
    st_idx, end_idx = _truncate_cell_index(len_list, MAX_CELL_CONTEXT_LENGTH)
    cell_ranges.append((st_idx, end_idx))
    cell_indices.append(len_list)
    for k in range(st_idx, end_idx):
      cur_line += context_data[i][k]
    pad_len = MAX_CELL_CONTEXT_LENGTH - len(cur_line)
    pad_lens.append(pad_len)
    cur_line += [0] * pad_len # [PAD]
    row_context += cur_line
  return row_context, pad_lens, cell_ranges, cell_indices


def col_context_builder(context_data):
  transpose = []
  for j in range(21):
    cur = []
    for i in range(21):
      cur.append(context_data[i][j])
    transpose.append(cur)
  col_header = list(transpose[10])
  return row_context_builder(transpose, col_header)

def mask_builder(pad_lens):
  mask = []
  for pad_len in pad_lens:
    mask += [1.0] * (MAX_CELL_CONTEXT_LENGTH - pad_len) + [0.0] * pad_len
  # print("mask %d" % np.array(mask).shape)
  return mask

def indices_builder(len_list, ranges, type):
  indices = []
  for i in range(len(ranges)):
    st_idx, end_idx = ranges[i]
    cur = [0] * st_idx + len_list[i][st_idx:end_idx] + [0] * (21 - end_idx)
    if type == "col":
      cur = [0] + cur
    indices += cur
  # print("indices %d" % np.array(indices).shape)
  return indices

def mask_per_cell_builder(cell_ranges, type):
  mask = []
  for (st_idx, end_idx) in cell_ranges:
    cur = [0.0] * st_idx + [1.0] * (end_idx - st_idx) + [0.0] * (21 - end_idx)
    if type == "col":
      cur = [0.0] + cur
    mask += cur
  # print("mask per cell %d" % np.array(mask).shape)
  return mask

def pad_formula(formula_id):
  formula_pad_len = FORMULA_LENGTH - len(formula_id)
  return   formula_id + [0] * formula_pad_len

def example2feature(id, data):
  formula = tf.convert_to_tensor(pad_formula(formula_ids[id]))
  row_context, pad_row, row_cells, row_indices = row_context_builder(data['context_data'].numpy(), data['context_header'].numpy())
  col_context, pad_col, col_cells, col_indices = col_context_builder(data['context_data'].numpy())
  row_mask = tf.convert_to_tensor(mask_builder(pad_row))
  col_mask = tf.convert_to_tensor(mask_builder(pad_col))
  segment_ids = [0] * MAX_CELL_CONTEXT_LENGTH + [1] * 21 * MAX_CELL_CONTEXT_LENGTH
  row_indices = tf.convert_to_tensor(indices_builder(row_indices, row_cells, "row"))
  col_indices = tf.convert_to_tensor(indices_builder(col_indices, col_cells, "col"))
  row_mask_per_cell = tf.convert_to_tensor(mask_per_cell_builder(row_cells, "row"))
  col_mask_per_cell = tf.convert_to_tensor(mask_per_cell_builder(col_cells, "col"))
  row_segment_ids_per_cell = tf.convert_to_tensor([0] * 21 + [1] * 21 * 21)
  col_segment_ids_per_cell = tf.convert_to_tensor([0] * 22 + [1] * 21 * 22)
  record_index = tf.convert_to_tensor([data['record_index'].numpy()])
  col_index = tf.convert_to_tensor([data['col_index'].numpy()])

  serialized_formula = tf.io.serialize_tensor(formula)
  serialized_row_context = tf.io.serialize_tensor(row_context)
  serialized_col_context = tf.io.serialize_tensor(col_context)
  serialized_row_mask = tf.io.serialize_tensor(row_mask)
  serialized_col_mask = tf.io.serialize_tensor(col_mask)
  serialized_segment_ids = tf.io.serialize_tensor(segment_ids)
  serialized_row_indices = tf.io.serialize_tensor(row_indices)
  serialized_col_indices = tf.io.serialize_tensor(col_indices)
  serialized_row_mask_per_cell = tf.io.serialize_tensor(row_mask_per_cell)
  serialized_col_mask_per_cell = tf.io.serialize_tensor(col_mask_per_cell)
  serialized_row_segment_ids_per_cell = tf.io.serialize_tensor(row_segment_ids_per_cell)
  serialized_col_segment_ids_per_cell = tf.io.serialize_tensor(col_segment_ids_per_cell)
  serialized_record_index = tf.io.serialize_tensor(record_index)
  serialized_col_index = tf.io.serialize_tensor(col_index)

  feature = {
          'formula': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_formula.numpy()])),
          'row_cell_context': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_row_context.numpy()])),
          'col_cell_context': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_col_context.numpy()])),
          'row_context_mask': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_row_mask.numpy()])),
          'col_context_mask': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_col_mask.numpy()])),
          'row_context_segment_ids': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_segment_ids.numpy()])),
          'col_context_segment_ids': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_segment_ids.numpy()])),
          'row_cell_indices': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_row_indices.numpy()])),
          'col_cell_indices': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_col_indices.numpy()])),
          'row_context_mask_per_cell': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_row_mask_per_cell.numpy()])),
          'col_context_mask_per_cell': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_col_mask_per_cell.numpy()])),
          'row_context_segment_ids_per_cell': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_row_segment_ids_per_cell.numpy()])),
          'col_context_segment_ids_per_cell': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_col_segment_ids_per_cell.numpy()])),
          'record_index': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_record_index.numpy()])),
          'col_index': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_col_index.numpy()]))
          }
  return feature

In [43]:
!ls
!rm -rf datasets
!mkdir datasets
!touch datasets/train.tfrecords
!touch datasets/valid.tfrecords
!touch datasets/test.tfrecords

bert_modeling.py	model_utils.py
constants.py		__pycache__
datasets		README.md
enron			spreadsheetcoder_model_architecture.png
experiments.ipynb	tokenization.py
mobilebert_modeling.py	vocab.txt
model.py


In [44]:
data_dir = 'datasets'
train_tfrecord = data_dir + '/train.tfrecords'
valid_tfrecord = data_dir + '/valid.tfrecords'
test_tfrecord = data_dir + '/test.tfrecords'

MAX_CELL_CONTEXT_LENGTH = 128

def record_data(file_name, st_idx, end_idx):
  with tf.io.TFRecordWriter(file_name) as writer:
    for i, data in enumerate(format_dataset[st_idx:end_idx]):
      feature = example2feature(i + st_idx, data)
      example = tf.train.Example(features=tf.train.Features(feature=feature))
      writer.write(example.SerializeToString())


TRAIN_SIZE = 130000
VALID_SIZE = 25000

record_data(train_tfrecord, 0, TRAIN_SIZE)
record_data(valid_tfrecord, TRAIN_SIZE, TRAIN_SIZE + VALID_SIZE)
record_data(test_tfrecord, TRAIN_SIZE + VALID_SIZE, TOTAL_SIZE)
# record_data(train_tfrecord, 0, 1)
# record_data(valid_tfrecord, 1, 2)
# record_data(test_tfrecord, 2, 3)

mask 2816
mask 2816
indices 462
indices 484
mask per cell 462
mask per cell 484
mask 2816
mask 2816
indices 462
indices 484
mask per cell 462
mask per cell 484
mask 2816
mask 2816
indices 462
indices 484
mask per cell 462
mask per cell 484


In [45]:
tf.compat.v1.disable_eager_execution()

BATCH_SIZE = 64

feature_description = {
    'formula': tf.io.FixedLenFeature([], tf.string),
    'row_cell_context': tf.io.FixedLenFeature([], tf.string),
    'col_cell_context': tf.io.FixedLenFeature([], tf.string),
    'row_context_mask': tf.io.FixedLenFeature([], tf.string),
    'col_context_mask': tf.io.FixedLenFeature([], tf.string),
    'row_context_segment_ids': tf.io.FixedLenFeature([], tf.string),
    'col_context_segment_ids': tf.io.FixedLenFeature([], tf.string),
    'row_cell_indices': tf.io.FixedLenFeature([], tf.string),
    'col_cell_indices': tf.io.FixedLenFeature([], tf.string),
    'row_context_mask_per_cell': tf.io.FixedLenFeature([], tf.string),
    'col_context_mask_per_cell': tf.io.FixedLenFeature([], tf.string),
    'row_context_segment_ids_per_cell': tf.io.FixedLenFeature([], tf.string),
    'col_context_segment_ids_per_cell': tf.io.FixedLenFeature([], tf.string),
    'record_index': tf.io.FixedLenFeature([], tf.string),
    'col_index': tf.io.FixedLenFeature([], tf.string),
}

train_raw = tf.data.TFRecordDataset(train_tfrecord)
valid_raw = tf.data.TFRecordDataset(valid_tfrecord)
test_raw = tf.data.TFRecordDataset(test_tfrecord)

def _parse_example(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  feature_dict = tf.io.parse_single_example(example_proto, feature_description)
  feature_dict['formula'] = tf.io.parse_tensor(feature_dict['formula'], tf.int32)
  feature_dict['row_cell_context'] = tf.io.parse_tensor(feature_dict['row_cell_context'], tf.int32)
  feature_dict['col_cell_context'] = tf.io.parse_tensor(feature_dict['col_cell_context'], tf.int32)
  feature_dict['row_context_mask'] = tf.io.parse_tensor(feature_dict['row_context_mask'], tf.float32)
  feature_dict['col_context_mask'] = tf.io.parse_tensor(feature_dict['col_context_mask'], tf.float32)
  feature_dict['row_context_segment_ids'] = tf.io.parse_tensor(feature_dict['row_context_segment_ids'], tf.int32)
  feature_dict['col_context_segment_ids'] = tf.io.parse_tensor(feature_dict['col_context_segment_ids'], tf.int32)
  feature_dict['row_cell_indices'] = tf.io.parse_tensor(feature_dict['row_cell_indices'], tf.int32)
  feature_dict['col_cell_indices'] = tf.io.parse_tensor(feature_dict['col_cell_indices'], tf.int32)
  feature_dict['row_context_mask_per_cell'] = tf.io.parse_tensor(feature_dict['row_context_mask_per_cell'], tf.float32)
  feature_dict['col_context_mask_per_cell'] = tf.io.parse_tensor(feature_dict['col_context_mask_per_cell'], tf.float32)
  feature_dict['row_context_segment_ids_per_cell'] = tf.io.parse_tensor(feature_dict['row_context_segment_ids_per_cell'], tf.int32)
  feature_dict['col_context_segment_ids_per_cell'] = tf.io.parse_tensor(feature_dict['col_context_segment_ids_per_cell'], tf.int32)
  feature_dict['record_index'] = tf.io.parse_tensor(feature_dict['record_index'], tf.int32)
  feature_dict['col_index'] = tf.io.parse_tensor(feature_dict['col_index'], tf.int32)
  return feature_dict

train_data = train_raw.map(_parse_example)
valid_data = valid_raw.map(_parse_example)
test_data = test_raw.map(_parse_example)

In [46]:
train_data = train_data.shuffle(buffer_size=2000)
train_data = train_data.repeat()
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(tf.data.experimental.AUTOTUNE)

valid_data = valid_data.batch(BATCH_SIZE)

test_data = test_data.batch(BATCH_SIZE)

# Model![model architecture](https://raw.github.com/wy-go/google-research/master/spreadsheet_coder/spreadsheetcoder_model_architecture.png)


In [47]:
from model import create_model
from model_utils import get_train_op
from bert_modeling import BertConfig

NUM_ENCODER_LAYERS = 1 # not provided in paper
NUM_DECODER_LAYERS = 1 
EMBEDDING_SIZE = 512 # not provided in paper
HIDDEN_SIZE = 512
DROPOUT_RATE = 0.1
BEAM_SIZE = 64

BERT_CONFIG = BertConfig(hidden_size=512, hidden_act='gelu', 
                         initializer_range=0.02, vocab_size=30522, 
                         hidden_dropout_prob=0.1, num_attention_heads=8,
                         type_vocab_size=2, max_position_embeddings=512,
                         num_hidden_layers=8, intermediate_size=2048,
                         attention_probs_dropout_prob=0.1)  # BERT-Medium

formula_placeholder = tf.compat.v1.placeholder(name='formula', shape=(BATCH_SIZE, FORMULA_LENGTH), dtype=tf.int32)
row_cell_context_placeholder = tf.compat.v1.placeholder(name='row_cell_context', shape=(BATCH_SIZE, 22*MAX_CELL_CONTEXT_LENGTH), dtype=tf.int32) 
col_cell_context_placeholder = tf.compat.v1.placeholder(name='col_cell_context', shape=(BATCH_SIZE, 22*MAX_CELL_CONTEXT_LENGTH), dtype=tf.int32) 
row_context_mask_placeholder = tf.compat.v1.placeholder(name='row_context_mask', shape=(BATCH_SIZE, 22*MAX_CELL_CONTEXT_LENGTH), dtype=tf.float32) 
col_context_mask_placeholder = tf.compat.v1.placeholder(name='col_context_mask', shape=(BATCH_SIZE, 22*MAX_CELL_CONTEXT_LENGTH), dtype=tf.float32) 
row_context_segment_ids_placeholder = tf.compat.v1.placeholder(name='row_context_segment_ids', shape=(BATCH_SIZE, 22*MAX_CELL_CONTEXT_LENGTH), dtype=tf.int32) 
col_context_segment_ids_placeholder = tf.compat.v1.placeholder(name='col_context_segment_ids', shape=(BATCH_SIZE, 22*MAX_CELL_CONTEXT_LENGTH), dtype=tf.int32) 
row_cell_indices_placeholder = tf.compat.v1.placeholder(name='row_cell_indices', shape=(BATCH_SIZE, 22*21), dtype=tf.int32) 
col_cell_indices_placeholder = tf.compat.v1.placeholder(name='col_cell_indices', shape=(BATCH_SIZE, 22*22), dtype=tf.int32) 
row_context_mask_per_cell_placeholder = tf.compat.v1.placeholder(name='row_context_mask_per_cell', shape=(BATCH_SIZE, 22*21), dtype=tf.float32) 
col_context_mask_per_cell_placeholder = tf.compat.v1.placeholder(name='col_context_mask_per_cell', shape=(BATCH_SIZE, 22*22), dtype=tf.float32) 
row_context_segment_ids_per_cell_placeholder = tf.compat.v1.placeholder(name='row_context_segment_ids_per_cell', shape=(BATCH_SIZE, 22*21), dtype=tf.int32) 
col_context_segment_ids_per_cell_placeholder = tf.compat.v1.placeholder(name='col_context_segment_ids_per_cell', shape=(BATCH_SIZE, 22*22), dtype=tf.int32) 
record_index_placeholder = tf.compat.v1.placeholder(name='record_index',shape=(BATCH_SIZE, 1), dtype=tf.int32) 
col_index_placeholder = tf.compat.v1.placeholder(name='col_index',shape=(BATCH_SIZE, 1), dtype=tf.int32) 

bert_modeling:
- get_shape_list()
- BertModel()
- gelu()

model_utils:
- configure_tpu(flags): Configures the TPU from the command line flags.
- class AdamWeightDecayOptimizer(): apply_gradients(grads_and_vars, global_step=None, name=None)
- get_train_op(flags, total_loss, ema=None, tvars=None): Generates the training operation.
- construct_scalar_host_call(monitor_dict, model_dir, prefix="", reduce_fn=None): Construct host calls to monitor training progress on TPUs.
- build_lstm(num_units)
- print_tensors(**tensors): Host call function to print Tensors from the TPU during training.
- get_assignment_map_from_checkpoint(vars_to_restore, init_checkpoint, bert_prefix=""): Compute the union of the current variables and checkpoint variables.

# Experiment

  <table border="1">
    <caption><i>Table 4. </i>Hyper-parameters</caption>
    <thead>
      <tr align=center>
        <th></th>
        <th></th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>#Encoder layer</th>
        <td></td>
      </tr>
      <tr align=center>
        <th align=left>#Decoder layer</th>
        <td>1</td>
      </tr>
      <tr align=center>
        <th align=left>Embedding size</th>
        <td></td>
      </tr>
      <tr align=center>
        <th align=left>Hidden size</th>
        <td>512</td>
      </tr>
      <tr align=center>
        <th align=left>Dropout rate</th>
        <td>0.1</td>
      </tr>
      <tr align=center>
        <th align=left>Initial lr</th>
        <td>5e-5</td>
      </tr>
      <tr align=center>
        <th align=left>Batch size</th>
        <td>64</td>
      </tr>
      <tr align=center>
        <th align=left>Gradient clipping norm</th>
        <td>1.0</td>
      </tr>
      <tr align=center>
        <th align=left>Optimizer</th>
        <td>Adam</td>
      </tr>
      <tr align=center>
        <th align=left>#Minibatch updates</th>
        <td>200K</td>
      </tr>
      <tr align=center>
        <th align=left>Beam size</th>
        <td>64</td>
      </tr>
    </tbody>
  </table>

## Full model

In [48]:
y_train = create_model(num_encoder_layers=NUM_ENCODER_LAYERS,
                      num_decoder_layers=NUM_DECODER_LAYERS,
                      embedding_size=EMBEDDING_SIZE,
                      hidden_size=HIDDEN_SIZE,
                      dropout_rate=DROPOUT_RATE,
                      is_training=True,
                      formula=formula_placeholder, 
                      row_cell_context=row_cell_context_placeholder,
                      col_cell_context=col_cell_context_placeholder,
                      row_context_mask=row_context_mask_placeholder, 
                      col_context_mask=col_context_mask_placeholder,
                      row_context_segment_ids=row_context_segment_ids_placeholder,
                      col_context_segment_ids=col_context_segment_ids_placeholder,
                      row_cell_indices=row_cell_indices_placeholder,
                      col_cell_indices=col_cell_indices_placeholder,
                      row_context_mask_per_cell=row_context_mask_per_cell_placeholder,
                      col_context_mask_per_cell=col_context_mask_per_cell_placeholder,
                      row_context_segment_ids_per_cell=row_context_segment_ids_per_cell_placeholder,
                      col_context_segment_ids_per_cell=col_context_segment_ids_per_cell_placeholder,
                      exclude_headers=False,
                      max_cell_context_length=MAX_CELL_CONTEXT_LENGTH,
                      num_rows=21,
                      record_index=record_index_placeholder,
                      column_index=col_index_placeholder,
                      layer_norm=True,
                      cell_position_encoding=True,
                      cell_context_encoding=True,
                      use_bert=True,
                      use_mobilebert=False,
                      per_row_encoding=False,
                      max_pooling=True, #
                      use_cnn=True,
                      use_pointer_network=False, #
                      two_stage_decoding=True,
                      conv_type='cross',
                      grid_type='both',
                      skip_connection=True,
                      bert_config=BERT_CONFIG,
                      unused_tensors_to_print=True, # unused
                      formula_length=FORMULA_LENGTH,
                      formula_prefix_length=0,
                      vocab_size=VOCAB_SIZE,
                      beam_size=1,
                      use_tpu=True,
                      use_one_hot_embeddings=False
                      )


y_test, y_p = create_model(num_encoder_layers=NUM_ENCODER_LAYERS,
                           num_decoder_layers=NUM_DECODER_LAYERS,
                           embedding_size=EMBEDDING_SIZE,
                           hidden_size=HIDDEN_SIZE,
                           dropout_rate=DROPOUT_RATE,
                           is_training=False,
                           formula=formula_placeholder, 
                           row_cell_context=row_cell_context_placeholder,
                           col_cell_context=col_cell_context_placeholder,
                           row_context_mask=row_context_mask_placeholder, 
                           col_context_mask=col_context_mask_placeholder,
                           row_context_segment_ids=row_context_segment_ids_placeholder,
                           col_context_segment_ids=col_context_segment_ids_placeholder,
                           row_cell_indices=row_cell_indices_placeholder,
                           col_cell_indices=col_cell_indices_placeholder,
                           row_context_mask_per_cell=row_context_mask_per_cell_placeholder,
                           col_context_mask_per_cell=col_context_mask_per_cell_placeholder,
                           row_context_segment_ids_per_cell=row_context_segment_ids_per_cell_placeholder,
                           col_context_segment_ids_per_cell=col_context_segment_ids_per_cell_placeholder,
                           exclude_headers=False,
                           max_cell_context_length=MAX_CELL_CONTEXT_LENGTH,
                           num_rows=21,
                           record_index=record_index_placeholder,
                           column_index=col_index_placeholder,
                           layer_norm=True,
                           cell_position_encoding=True,
                           cell_context_encoding=True,
                           use_bert=True,
                           use_mobilebert=False,
                           per_row_encoding=False,
                           max_pooling=True, #
                           use_cnn=True,
                           use_pointer_network=False,
                           two_stage_decoding=True,
                           conv_type='cross',
                           grid_type='both',
                           skip_connection=True,
                           bert_config=BERT_CONFIG,
                           unused_tensors_to_print=True, # unused
                           formula_length=FORMULA_LENGTH,
                           formula_prefix_length=0,
                           vocab_size=VOCAB_SIZE,
                           beam_size=BEAM_SIZE,
                           use_tpu=True,
                           use_one_hot_embeddings=False
                           )

/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:393: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  warnings.warn('`tf.layers.dropout` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.
Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.


## Different encoder architecture
- — Column-based BERT
- — Row-based BERT
- — Convolution layers

### — Column-based BERT

### — Row-based BERT

### — Convolution layers

## Different decoding architecture
Same predictor for both the sketch and ranges, with a single output vocabulary

### — Two-stage decoding

## Different model initialization
Randomly initialize BERT encoders

### — Pretraining

## Previous approaches
Randomly initialize BERT encoders

### Row-based RobustFill

### Column-based RobustFill

## Baseline
LSTM decoder only

### No context

# Train

In [49]:
NUM_BATCHES = 200000

WU_STEPS = 1000 #
INIT_LR = 5e-5
NUM_EPOCH = 72
DECAY_METHOD = "poly" # or "cos"
WEIGHT_DECAY = 0.01
TRAIN_STEPS = 10000 #
MIN_LR_RATIO = 0.1 #
ADAM_EPS = 1e-8
NUM_CORE_PER_HOST = 1 #
CLIP = 1.0


class Flags:
  def __init__(self, flags_dic={}):
    if "warmup_steps" in flags_dic.keys():
      self.warmup_steps = flags_dic["warmup_steps"]
    else:
      self.warmup_steps = WU_STEPS
    if "learning_rate" in flags_dic.keys():
      self.learning_rate = flags_dic["learning_rate"]
    else:
      self.learning_rate = INIT_LR
    if "decay_method" in flags_dic.keys():
      self.decay_method = flags_dic["decay_method"]
    else:
      self.decay_method = DECAY_METHOD
    if "weight_decay" in flags_dic.keys():
      self.weight_decay = flags_dic["weight_decay"]
    else:
      self.weight_decay = WEIGHT_DECAY
    if "train_steps" in flags_dic.keys():
      self.train_steps = flags_dic["train_steps"]
    else:
      self.train_steps = TRAIN_STEPS
    if "min_lr_ratio" in flags_dic.keys():
      self.min_lr_ratio = flags_dic["min_lr_ratio"]
    else:
      self.min_lr_ratio = MIN_LR_RATIO
    if "adam_epsilon" in flags_dic.keys():
      self.adam_epsilon = flags_dic["adam_epsilon"]
    else:
      self.adam_epsilon = ADAM_EPS
    if "num_core_per_host" in flags_dic.keys():
      self.num_core_per_host = flags_dic["num_core_per_host"]
    else:
      self.num_core_per_host = NUM_CORE_PER_HOST #
    if "use_tpu" in flags_dic.keys():
      self.use_tpu = flags_dic["use_tpu"]
    else:
      self.use_tpu = False
    if "clip" in flags_dic.keys():
      self.clip = flags_dic["clip"]
    else:
      self.clip = CLIP

# FLAGS = tf.compat.v1.flags.FLAGS

# tf.compat.v1.flags.DEFINE_integer("warmup_steps", WU_STEPS, "warmup steps")
# tf.compat.v1.flags.DEFINE_float("learning_rate", INIT_LR, "learning rate")
# tf.compat.v1.flags.DEFINE_string("decay_method", DECAY_METHOD, "decay method")
# tf.compat.v1.flags.DEFINE_float("weight_decay", WEIGHT_DECAY, "weight decay")
# tf.compat.v1.flags.DEFINE_integer("train_steps", TRAIN_STEPS, "train steps")
# tf.compat.v1.flags.DEFINE_float("min_lr_ratio", MIN_LR_RATIO, "minimum learning rate ratio")
# tf.compat.v1.flags.DEFINE_float("adam_epsilon", ADAM_EPS, "adam epsilon")
# tf.compat.v1.flags.DEFINE_integer("num_core_per_host", NUM_CORE_PER_HOST, "number of core per host")
# tf.compat.v1.flags.DEFINE_bool("use_tpu", False, "whether use TPU")
# tf.compat.v1.flags.DEFINE_float("clip", CLIP, "gradient clipping norm")

FLAGS = Flags()

In [50]:
loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=formula_placeholder, y_pred=y_train)

train_op, learning_rate, gnorm = get_train_op(FLAGS, loss)

count = y_test.get_shape()[0] # batch_size
top1_value = tf.gather(y_test, tf.argmax(y_p, axis=-1, output_type=tf.int32), axis=1, batch_dims=1) # [batch_size, formula_length]
top1_value = tf.slice(top1_value, [0, 1], [count, FORMULA_LENGTH])
top1_true = tf.cast(tf.equal(formula_placeholder, top1_value), tf.int32) # [batch_size, formula_length]
top1_accuracy = tf.reduce_sum(tf.cast(tf.equal(tf.reduce_sum(top1_true, 1), tf.fill([count], y_test.get_shape()[-1])), tf.int32)) / count

INFO:tensorflow:Adam WD excludes row/bert/embeddings/LayerNorm/beta
INFO:tensorflow:Adam WD excludes row/bert/embeddings/LayerNorm/gamma
INFO:tensorflow:Adam WD excludes row/bert/encoder/layer_0/attention/self/query/bias
INFO:tensorflow:Adam WD excludes row/bert/encoder/layer_0/attention/self/key/bias
INFO:tensorflow:Adam WD excludes row/bert/encoder/layer_0/attention/self/value/bias
INFO:tensorflow:Adam WD excludes row/bert/encoder/layer_0/attention/output/dense/bias
INFO:tensorflow:Adam WD excludes row/bert/encoder/layer_0/attention/output/LayerNorm/beta
INFO:tensorflow:Adam WD excludes row/bert/encoder/layer_0/attention/output/LayerNorm/gamma
INFO:tensorflow:Adam WD excludes row/bert/encoder/layer_0/intermediate/dense/bias
INFO:tensorflow:Adam WD excludes row/bert/encoder/layer_0/output/dense/bias
INFO:tensorflow:Adam WD excludes row/bert/encoder/layer_0/output/LayerNorm/beta
INFO:tensorflow:Adam WD excludes row/bert/encoder/layer_0/output/LayerNorm/gamma
INFO:tensorflow:Adam WD exc

In [51]:
init_op = tf.compat.v1.global_variables_initializer()

# iterator = tf.compat.v1.data.Iterator.from_structure(tf.compat.v1.data.get_output_types(train_data),
#                                                      tf.compat.v1.data.get_output_shapes(train_data))
# next_element = iterator.get_next()
# train_init_op = iterator.make_initializer(train_data)
# test_init_op = iterator.make_initializer(test_data)

train_iter = tf.compat.v1.data.make_one_shot_iterator(train_data)
train_batch = train_iter.get_next()

test_iter =  tf.compat.v1.data.make_one_shot_iterator(test_data)
test_batch = test_iter.get_next()

In [52]:
print(tf.compat.v1.data.get_output_types(train_data))
print(tf.compat.v1.data.get_output_shapes(train_data))

{'col_cell_context': tf.int32, 'col_cell_indices': tf.int32, 'col_context_mask': tf.float32, 'col_context_mask_per_cell': tf.float32, 'col_context_segment_ids': tf.int32, 'col_context_segment_ids_per_cell': tf.int32, 'col_index': tf.int32, 'formula': tf.int32, 'record_index': tf.int32, 'row_cell_context': tf.int32, 'row_cell_indices': tf.int32, 'row_context_mask': tf.float32, 'row_context_mask_per_cell': tf.float32, 'row_context_segment_ids': tf.int32, 'row_context_segment_ids_per_cell': tf.int32}
{'col_cell_context': TensorShape(None), 'col_cell_indices': TensorShape(None), 'col_context_mask': TensorShape(None), 'col_context_mask_per_cell': TensorShape(None), 'col_context_segment_ids': TensorShape(None), 'col_context_segment_ids_per_cell': TensorShape(None), 'col_index': TensorShape(None), 'formula': TensorShape(None), 'record_index': TensorShape(None), 'row_cell_context': TensorShape(None), 'row_cell_indices': TensorShape(None), 'row_context_mask': TensorShape(None), 'row_context_mas

In [ ]:
top1_accurs = []

with tf.compat.v1.Session() as sess:
  #Initialize variables
  sess.run(init_op)

  # Train
  # sess.run(train_init_op)
  for batch_index in range(NUM_BATCHES):
    # next_elem = sess.run(next_element)
    next_elem = sess.run(train_batch)
    formula = next_elem['formula']
    row_cell_context = next_elem['row_cell_context']
    col_cell_context = next_elem['col_cell_context']
    row_context_mask = next_elem['row_context_mask']
    col_context_mask = next_elem['col_context_mask']
    row_context_segment_ids = next_elem['row_context_segment_ids']
    col_context_segment_ids = next_elem['col_context_segment_ids']
    row_cell_indices = next_elem['row_cell_indices']
    col_cell_indices = next_elem['col_cell_indices']
    row_context_mask_per_cell = next_elem['row_context_mask_per_cell']
    col_context_mask_per_cell = next_elem['col_context_mask_per_cell']
    row_context_segment_ids_per_cell = next_elem['row_context_segment_ids_per_cell']
    col_context_segment_ids_per_cell = next_elem['col_context_segment_ids_per_cell']
    record_index = next_elem['record_index']
    col_index = next_elem['col_index']
    
    print(formula)
    print(len(row_cell_context))
    print(row_cell_context)
    print(len(col_cell_context))
    print(col_cell_context)
    print(len(record_index ))
    print(record_index )

    _, loss_val = sess.run([train_op, loss], feed_dict={formula_placeholder: formula,
                                                      row_cell_context_placeholder: row_cell_context,
                                                      col_cell_context_placeholder: col_cell_context,
                                                      row_context_mask_placeholder: row_context_mask,
                                                      col_context_mask_placeholder: col_context_mask,
                                                      row_context_segment_ids_placeholder: row_context_segment_ids,
                                                      col_context_segment_ids_placeholder: col_context_segment_ids,
                                                      row_cell_indices_placeholder: row_cell_indices,
                                                      col_cell_indices_placeholder: col_cell_indices,
                                                      row_context_mask_per_cell_placeholder: row_context_mask_per_cell,
                                                      col_context_mask_per_cell_placeholder: col_context_mask_per_cell,
                                                      row_context_segment_ids_per_cell_placeholder: row_context_segment_ids_per_cell,
                                                      col_context_segment_ids_per_cell_placeholder: col_context_segment_ids_per_cell,
                                                      record_index_placeholder: record_index,
                                                      col_index_placeholder: col_index
                                                      })
    print("batch %d: loss %f" % (batch_index, loss_val))

  # Test
  sess.run(test_init_op)
  while True:
    try:
      # next_elem = sess.run(next_element)
      next_elem = sess.run(test_batch)
      formula = next_elem['formula']
      row_cell_context = next_elem['row_cell_context']
      col_cell_context = next_elem['col_cell_context']
      row_context_mask = next_elem['row_context_mask']
      col_context_mask = next_elem['col_context_mask']
      row_context_segment_ids = next_elem['row_context_segment_ids']
      col_context_segment_ids = next_elem['col_context_segment_ids']
      row_cell_indices = next_elem['row_cell_indices']
      col_cell_indices = next_elem['col_cell_indices']
      row_context_mask_per_cell = next_elem['row_context_mask_per_cell']
      col_context_mask_per_cell = next_elem['col_context_mask_per_cell']
      row_context_segment_ids_per_cell = next_elem['row_context_segment_ids_per_cell']
      col_context_segment_ids_per_cell = next_elem['col_context_segment_ids_per_cell']
      record_index = next_elem['record_index']
      column_index = next_elem['col_index']

      top1_accuracy_val = sess.run(top1_accuracy)
      top1_accurs.append(top1_accuracy_val)
    except tf.errors.OutOfRangeError:
      print('End of test datset.')
      break
  print("Top1 accuracy: %.2f" % np.mean(top1_accurs))


[[58 59 60 61 62  7 57 21 40 63 11 40  6  1  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [58 59 60 61 62  7 57 21 40 63 11 40  6  1  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]]
2
[[4064  102 4064 ...    0    0    0]
 [4064  102 4064 ...    0    0    0]]
2
[[4064  102 4064 ...    0    0    0]
 [4064  102 4064 ...    0    0    0]]
2
[[7]
 [7]]



# Results

  <table border="1">
  <caption><i>Table 5. </i>Full model formula accuracy from paper.</caption>
    <thead>
      <tr>
        <th align=left>Dataset</th>
        <th>Top-1</th>
        <th>Top-5</th>
        <th>Top-10</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>Enron</th>
        <td>29.8%</td>
        <td>41.8%</td>
        <td>48.5%</td>
      </tr>
      <tr align=center>
        <th align=left>Google Sheets</th>
        <td>42.51%</td>
        <td>54.41%</td>
        <td>58.57%</td>
      </tr>
    </tbody>
  </table>

  <br/>

  <table border="1">
    <caption><i>Table 6. </i>Formula accuracy on the Google Sheets test set.</caption>
    <thead>
      <tr>
        <th align=left>Approach</th>
        <th>Top-1</th>
        <th>Top-5</th>
        <th>Top-10</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>Full model</th>
        <td><b>42.51%</b></td>
        <td><b>54.41%</b></td>
        <td><b>58.57%</b></td>
      </tr>
      <tr align=center>
        <th align=left>— Column-based BERT</th>
        <td>39.42%</td>
        <td>51.68%</td>
        <td>56.50%</td>
      </tr>
      <tr align=center>
        <th align=left>— Row-based BERT</th>
        <td>20.37%</td>
        <td>40.87%</td>
        <td>48.37%</td>
      </tr>
      <tr align=center>
        <th align=left>— Convolution layers</th>
        <td>38.43%</td>
        <td>51.31%</td>
        <td>55.87%</td>
      </tr>
      <tr align=center>
        <th align=left>— Two-stage decoding</th>
        <td>41.12%</td>
        <td>53.57%</td>
        <td>57.95%</td>
      </tr>
      <tr align=center>
        <th align=left>— Pretraining</th>
        <td>31.51%</td>
        <td>42.64%</td>
        <td>49.77%</td>
      </tr>
      <tr align=center>
        <th align=left>Row-based RobustFill</th>
        <td>31.14%</td>
        <td>40.09%</td>
        <td>47.10%</td>
      </tr>
      <tr align=center>
        <th align=left>Column-based RobustFill</th>
        <td>20.65%</td>
        <td>39.69%</td>
        <td>46.96%</td>
      </tr>
      <tr align=center>
        <th align=left>No context</th>
        <td>10.56%</td>
        <td>23.27%</td>
        <td>31.96%</td>
      </tr>
    </tbody>
  </table>